In [71]:
# This R environment comes with many helpful analytics packages installed
# It is defined by the kaggle/rstats Docker image: https://github.com/kaggle/docker-rstats
# For example, here's a helpful package to load

library(tidyverse) # metapackage of all tidyverse packages
library(magrittr)
library(glmnet)
library(caret)
library(randomForest)
library(Metrics)
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

list.files(path = "../input")

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

[1] "the-price-of-art"

**Updated**
I had to rewrite the code here to include the obectID column (in the previous attempt I had removed it, but was having difficulty submitting the file with the removed column. To work with it here, I had to remove the column from the analyses. 

In [72]:
test<-read.csv("/kaggle/input/the-price-of-art/test.csv")
train<-read.csv("/kaggle/input/the-price-of-art/train.csv")
sample<-read.csv("/kaggle/input/the-price-of-art/sample.csv")
exchange<-read.csv("/kaggle/input/the-price-of-art/exchange_rates.csv")

In [73]:
#New variables for literature not used in HW1
train <- train %>% mutate(literature = case_when(is.na(train$literature) ~ 0, TRUE ~ 1))


target_painting<-"oil"

train$new_painting<-as.integer(grepl(target_painting,train$details))


In the previous assignment, I didnt use literature and category, so I will start by making these two variables usable for this assignment. 

In [74]:
#ditto for test
target_painting<- "oil"

test$new_painting<-as.integer(grepl(target_painting,test$details))

test <- test %>% mutate(literature = case_when(is.na(test$literature) ~ 0, TRUE ~ 1))


In [67]:
#fixing measurement columns
train$height_cm <- as.numeric(train$height_cm)
train$width_cm <- as.numeric(train$width_cm)

In [69]:
#ditto for test
test$height_cm <- as.numeric(test$height_cm)
test$width_cm <- as.numeric(test$width_cm)

In [77]:
#Pulling in variables from HW 1

train <- train %>%
mutate(
new_exhibited = case_when(is.na(train$exhibited) ~ 0, TRUE ~ 1))

train <- train %>%
mutate(
new_provenance = case_when(is.na(train$provenance) ~ 0, TRUE ~ 1)) 

In [78]:
#ditto for test

test <- test %>%
mutate(
new_exhibited = case_when(is.na(test$exhibited) ~ 0, TRUE ~ 1))

test <- test %>%
mutate(
new_provenance = case_when(is.na(test$provenance) ~ 0, TRUE ~ 1)) 

In [79]:
head(train)

,object_id,artist,original_currency,height_cm,width_cm,auction,location,details,provenance,literature,exhibited,category,date,estimate_low_usd,estimate_high_usd,price_realized_usd,new_painting,new_exhibited,new_provenance
,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<int>,<dbl>,<dbl>
1,vgfmo,MR DOODLE (B. 1994),HKD,173,122,Mr Doodle: Caravan Chaos,Hong Kong,MR DOODLE (B. 1994) Caravan Chaos: Side 2 Panel 4 spray paint and sticker on aluminium panel mounted on plywood 173 x 122 cm. (68 1/8 x 48 in.) Executed in 2015,"Private collection, Europe (Acquired directly from the artist) Private collection, Europe (Acquired from the above by the present owner)",0,NA,Paintings,2021-05-24,12877.97,25755.94,38633.91,0,0,1
2,lgvfk,Mario Merz (1925-2003),GBP,126,243,The Italian Sale,London,"Mario Merz (1925-2003) Meteoriti in giardino (Meteorites in the Garden) signed, dated and titled ‘Mario Merz 76 Meteoriti in giardino’ (lower right) charcoal, clay and snails on canvas 49 5/8 x 95 5/8in. (126 x 243cm.) Executed in 1976","Galleria Salvatore + Caroline Ala, Milan. Acquired from the above by the family of the present owner.",0,NA,Paintings,2016-10-06,126470.00,189705.00,279498.70,0,0,1
3,svvhb,"Attributed to Julius Caesar Ibbetson (Leeds 1759-1817 Masham, N. Yorks)",USD,46.3,60.9,Old Master Paintings Part II,New York,"Attributed to Julius Caesar Ibbetson (Leeds 1759-1817 Masham, N. Yorks) Landscape with a traveler on horseback and ruins in the distance oil on canvas 18¼ x 24 in. (46.3 x 60.9 cm.)","Anonymous sale; Sotheby's, London, 6 June 1962, lot 183 (£90 to Agnew). with Agnew's, London. Anonymous sale; Christie's, London, 18 March 1988, lot 187 (£900).",0,NA,Paintings,2013-01-31,8000.00,10000.00,7500.00,1,0,1
4,zzevu,"Mortimer Luddington Menpes, R.I., R.B.A., R.E. (Port Adelaide 1855-1938 Pangbourne)",GBP,31.6,26,"Glebe House, Mont Pellier and Woodbury House: Three Country House Collections",London (South Kensigton),"Mortimer Luddington Menpes, R.I., R.B.A., R.E. (Port Adelaide 1855-1938 Pangbourne) Japanese musicians in an interior with a shamisen signed 'Mortimer Menpes.' (lower left) oil on board 12 ½ x 10 ¼ in. (31.6 x 26 cm.)",NA,0,NA,Paintings,2015-06-17,2356.20,3927.00,7461.30,1,0,0
5,iitho,Ed Ruscha (b. 1937),GBP,142.3,340.5,Post-War and Contemporary Art Evening Auction,London,"Ed Ruscha (b. 1937) Ship Talk signed and dated 'Ed Ruscha 1988' (on the reverse); signed, titled and dated 'ED RUSCHA 1988 ""SHIP TALK""' (on the stretcher) acrylic on canvas 56 x 134 1/8in. (142.3 x 340.5cm.) Painted in 1988","Mos Food Services, Tokyo. Paul Rusconi, Los Angeles. Gagosian Gallery, Los Angeles. Sandroni Rey, Los Angeles. Andrew and Lea Fastow, Houston. Anon. sale, Christie's New York, 13 November 2007, lot 66. Acquired at the above sale by the present owner.",1,"Nagoya, Institute of Contemporary Arts, Edward Ruscha, 1988-1989, p. 52, pl. 25 (illustrated in colour, pp. 42-43). This exhibition later travelled to Tokyo, Touko Museum of Contemporary Art. Victoria, Museum of Modern Art at Heide, Downtown: Ruscha, Rooney, Arkley, 1995 (illustrated in colour, p. 15). Houston, Menil Collection, 2001 (on extended loan).",Paintings,2012-02-13,1025700.00,1341300.00,1213876.50,0,1,1
6,mdreb,Morgan Russell (1886-1953),USD,32.4,24.1,American Art,New York,Morgan Russell (1886-1953) Synchromy oil on canvas laid down on board 12¾ x 9½ in. (32.4 x 24.1 cm.) Painted circa 1913.,"The artist. Estate of the above. Louis Sol, acquired from the above, 1955. Sale: Calmels, Chambre & Cohen, Paris, 6 April 2001, lot 64. Acquired by the present owner from the above.",0,NA,Paintings,2013-05-23,60000.00,80000.00,62500.00,1,0,1


In [80]:
train1<-train[,c(1, 14, 15, 16, 17, 18, 19)]
sapply(train1, anyNA)

object_id   estimate_low_usd  estimate_high_usd price_realized_usd 
             FALSE              FALSE              FALSE              FALSE 
      new_painting      new_exhibited     new_provenance 
             FALSE              FALSE              FALSE

In [81]:
#ditto for test
test1<-test[,c(1, 14, 15, 16, 17, 18)]
test1<- na.omit(test1)
head(test1)

,object_id,estimate_low_usd,estimate_high_usd,new_painting,new_exhibited,new_provenance
,<chr>,<dbl>,<dbl>,<int>,<dbl>,<dbl>
1,hxjim,6407.273,10251.64,0,0,0
2,ouyzj,278.640,417.96,0,0,0
3,vztxu,40000.000,60000.00,1,0,1
4,dbwnd,1000000.000,1500000.00,1,1,1
5,dmrus,14581.000,29162.00,1,0,0
6,zguak,10322.447,15483.67,0,0,0


In [85]:
train<-train1[, c(4, 1, 2, 3, 5, 6, 7)]
head(train)

,price_realized_usd,object_id,estimate_low_usd,estimate_high_usd,new_painting,new_exhibited,new_provenance
,<dbl>,<chr>,<dbl>,<dbl>,<int>,<dbl>,<dbl>
1,38633.91,vgfmo,12877.97,25755.94,0,0,1
2,279498.70,lgvfk,126470.00,189705.00,0,0,1
3,7500.00,svvhb,8000.00,10000.00,1,0,1
4,7461.30,zzevu,2356.20,3927.00,1,0,0
5,1213876.50,iitho,1025700.00,1341300.00,0,1,1
6,62500.00,mdreb,60000.00,80000.00,1,0,1


In [98]:
#the models had trouble running these variables in dbl quality, converting to integer. 
train$new_exhibited<-sapply(train$new_exhibited, as.integer)
train$new_provenance<-sapply(train$new_provenance, as.integer)
test1$new_exhibited<-sapply(test1$new_exhibited, as.integer)
test1$new_provenance<-sapply(test1$new_provenance, as.integer)

In [101]:
head(train)

,price_realized_usd,object_id,estimate_low_usd,estimate_high_usd,new_painting,new_exhibited,new_provenance
,<dbl>,<chr>,<dbl>,<dbl>,<int>,<int>,<int>
1,38633.91,vgfmo,12877.97,25755.94,0,0,1
2,279498.70,lgvfk,126470.00,189705.00,0,0,1
3,7500.00,svvhb,8000.00,10000.00,1,0,1
4,7461.30,zzevu,2356.20,3927.00,1,0,0
5,1213876.50,iitho,1025700.00,1341300.00,0,1,1
6,62500.00,mdreb,60000.00,80000.00,1,0,1


In [103]:
set.seed(123)

train_idx<- sample(nrow(train), nrow(train)*0.7)

train_data<-train[train_idx,]
test_data<-train[-train_idx, ]

#standarzie
x_train<-scale(train_data[, 3:7])
y_train<-train_data[,1]
x_test<-scale(test_data[, 3:7])
y_test<-test_data[,1]

# training the model
lambda_seq<- 10^seq(-2, 2, length = 100)
ridge_cv<- cv.glmnet(x_train, y_train, alpha = 0, lambda = lambda_seq)
lambda_opt<- ridge_cv$lamdba.min

#fit
ridge_fit<-glmnet(x_train, y_train, alpha=0, lambda=lambda_opt)
#predict
ridge_pred<-predict(ridge_fit, s=0.1, newx=x_test)
#rmsle
rmsle<-rmsle(ridge_pred, y_test)
rmsle


Warning message in log(1 + actual):
“NaNs produced”


[1] NaN

The rmsle value is roughly 1.6.

In [35]:
train1$price_realized_usd<-sapply(train1$price_realized_usd, as.integer)
head(train1)

,price_realized_usd,estimate_low_usd,estimate_high_usd,new_painting,area_cm,new_exhibited,new_provenance
,<int>,<dbl>,<dbl>,<int>,<dbl>,<int>,<int>
1,38633,12877.97,25755.94,0,21106.00,0,1
2,279498,126470.00,189705.00,0,30618.00,0,1
3,7500,8000.00,10000.00,1,2819.67,0,1
4,7461,2356.20,3927.00,1,821.60,0,0
5,1213876,1025700.00,1341300.00,0,48453.15,1,1
6,62500,60000.00,80000.00,1,780.84,0,1


In [102]:
#Cross validation
lambda_seq<-10^seq(-2, 2, length = 100)
lasso_cv<-cv.glmnet(x_train, y_train, alpha = 1, lambda = lambda_seq)

#OV
lambda_opt<-lasso_cv$lambda.min

#fitting model
lasso_model<-glmnet(x_train, y_train, alpha = 1, lambda = lambda_opt)

#preditction
x_test<-as.matrix(test_data[,(3:7)])
y_pred<-predict(lasso_model, newx = x_test)
y_test <- test_data$price_realized_usd

#rmsle
rmsle_value <- rmsle(y_test, y_pred)
rmsle


[1] NaN

I tried running this model a few different ways, getting a very high rmsle value each time. 

In [95]:
set.seed(123)

# Fitting the Random Forest model
rf_model <- randomForest(x = x_train, y = y_train)

# Predicting on the test set
x_test <- test_set[, c("estimate_low_usd", "estimate_high_usd", "new_painting", "area_cm", "new_exhibited", "new_provenance")]
y_test <- test_set$price_realized_usd
y_pred <- predict(rf_model, newdata = x_test)

# RMSLE
rmsle <- function(actual, predicted) {
  log_diff <- log1p(predicted) - log1p(actual)
  rmsle <- sqrt(mean(log_diff^2))
  return(rmsle)
}


rmsle_score <- rmsle(y_test, y_pred)

# Print
print(rmsle_score)

ERROR: Error in predict.randomForest(rf_model, newdata = x_test): missing values in newdata


In [55]:
set.seed(123)
#Final predictions and submit
final_model<-randomForest(x_train, y_train)

#final prediction
final_predict<- predict(final_model, newdata = x_test)


final_df <- data.frame(object_id = test$object_id, price_realized_usd = final_predict)

head(final_df)


It took a while to run in comparison to other models, but the output for this model was 1.09, making it the best fitting model. 

In [ ]:
write_csv(final_df, "submission.csv")